# 🧩 Capítulo 8 – Detección de Anomalías con Autoencoders

En este capítulo se introduce el enfoque de **Autoencoders**, uno de los primeros modelos de **Deep Learning** aplicados a la detección de anomalías.  

Su principal fortaleza radica en que **aprenden una representación comprimida (codificación) de los datos normales y luego intentan reconstruirlos**.  
Cuando el modelo encuentra un dato anómalo —algo que no encaja en el patrón aprendido— el **error de reconstrucción** tiende a ser mayor, lo que permite identificar la anomalía.

---

## 🎯 ¿Por qué Autoencoders?

La elección de Autoencoders responde a varias motivaciones:

- Son capaces de **modelar relaciones no lineales complejas** en los datos.  
- No requieren etiquetas, solo datos representativos de comportamiento normal.  
- Pueden incorporar **estructuras temporales** (mediante secuencias de entrada o variantes recurrentes).  
- Funcionan bien como paso previo a modelos más avanzados (ej. LSTM Autoencoders).  
- Representan una primera aproximación a las capacidades del **Deep Learning** en la detección de anomalías.

---

## 🧭 Objetivo de este capítulo

A lo largo del capítulo se abordarán:

- Los fundamentos teóricos de los Autoencoders y su uso para detección de anomalías.  
- La construcción de un **modelo base** entrenado sobre datos sintéticos y MDF.  
- El análisis del **error de reconstrucción** como score de anomalía.  
- La comparación cualitativa con los modelos clásicos (PCA, Isolation Forest, One-Class SVM).  

Además, se discutirán las principales variantes de Autoencoders y se introducirán las bases de su extensión hacia arquitecturas recurrentes como el **LSTM Autoencoder**.

---

## 🔍 Intuición

El Autoencoder es una red neuronal con dos partes:

- **Encoder**: comprime los datos de entrada en una representación de menor dimensión (*latent space*).  
- **Decoder**: intenta reconstruir la entrada a partir de esa representación comprimida.  

De forma intuitiva:  

> "Un Autoencoder aprende a copiar los datos normales. Cuando recibe algo raro (anómalo), no sabe reconstruirlo bien y el error de reconstrucción aumenta."

---

## 🧠 Fundamento Teórico

El entrenamiento de un Autoencoder se plantea como la minimización del error de reconstrucción:

$$
\min_{\theta} \; L(x, g_{\theta}(f_{\theta}(x)))
$$

Donde:

- $ f_{\theta}(x) $: encoder (codificación al espacio latente).  
- $ g_{\theta}(z) $: decoder (reconstrucción a partir del latente $z$).  
- $ L(\cdot)$: función de pérdida, normalmente el **MSE** (Mean Squared Error).  

El **score de anomalía** para un punto $x$ se define como:

$$
s(x) = \|x - \hat{x}\|^2
$$

Si $s(x)$ supera un umbral, el punto se clasifica como anómalo.

---

## 📐 Función de Decisión y Scores

- **Salida del modelo**: reconstrucción $\hat{x}$ de la entrada $x$.  
- **Score de anomalía**: error de reconstrucción.  
  - Error bajo → dato normal.  
  - Error alto → posible anomalía.

---

## ⚙️ Parámetros Clave

- **Dimensión del espacio latente**: controla la compresión (similar al número de componentes en PCA).  
- **Número de capas y neuronas**: mayor capacidad para aprender relaciones complejas.  
- **Función de activación**: ReLU, tanh, etc.  
- **Función de pérdida**: MSE, MAE o variantes robustas.  
- **Regularización**: dropout, weight decay, sparsity para evitar sobreajuste.

---

## ✅ Ventajas

- Gran capacidad para modelar relaciones no lineales.  
- Se adaptan bien a problemas de alta dimensión.  
- Escalables a grandes volúmenes de datos.  
- Flexibles: pueden extenderse a variantes recurrentes (LSTM, GRU) o convolucionales.

---

## 🚫 Limitaciones

- Requieren mayor poder computacional y tiempo de entrenamiento.  
- Mayor complejidad en la configuración (arquitectura, optimizador, regularización).  
- Pueden sobreajustar si se entrenan demasiado tiempo o con datos ruidosos.  
- El umbral para anomalías debe definirse cuidadosamente a partir de la distribución del error.

---

### 🔧 Ejemplo concreto

Supongamos que queremos monitorizar las **vibraciones de un motor** registradas por tres sensores.  

- En condiciones normales, las tres señales están correlacionadas: cuando una aumenta, las otras siguen un patrón consistente.  
- Entrenamos un Autoencoder para que aprenda estas relaciones y reconstruya las tres señales a la vez.  

Ahora, si en un momento dado el motor empieza a vibrar de forma irregular en uno de los ejes, el Autoencoder **no podrá reconstruir correctamente esa señal**, ya que rompe la correlación aprendida.  

El error de reconstrucción en ese punto será mucho mayor que en condiciones normales, permitiendo etiquetarlo como anómalo.

---

## 🔗 Referencias

Hinton, Geoffrey E., and Ruslan R. Salakhutdinov.  
**"Reducing the dimensionality of data with neural networks."** Science 313.5786 (2006): 504-507.  

Sakurada, Mayu, and Takehisa Yairi.  
**"Anomaly detection using autoencoders with nonlinear dimensionality reduction."** Proceedings of the MLSDA 2014 2nd workshop on machine learning for sensory data analysis. 2014.  

Zong, Bo, et al.  
**"Deep autoencoding Gaussian mixture model for unsupervised anomaly detection."** ICLR 2018.  
